## Librerías Adicionales

- `numpy`: Usada para operaciones matemáticas y manejo de arreglos en Python.
- `tensorflow.keras.models`: Facilita la carga de modelos previamente entrenados de Keras.
- `sklearn.preprocessing`: Proporciona herramientas adicionales para la codificación y el preprocesamiento de datos.

In [ ]:
pip install tensorflow

In [4]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import load_model
from sklearn.preprocessing import LabelEncoder
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

## Cargar el Modelo en el Notebook de Producción:

In [5]:
model = load_model('training_model_try_recommender.h5')

## Función de Recomendación: 
Esta función tomará la información del usuario como entrada y utilizará el modelo para predecir las calificaciones de los libros que el usuario aún no ha calificado. Luego, seleccionará los libros con las calificaciones más altas pronosticadas.

Aquí hay un ejemplo de cómo podría ser esta función para un usuario dandole 5 libros a recomendar:

In [8]:
# Cargar los datos
train_usuarios_calificaciones = pd.read_csv('train_usuarios_calificaciones.csv', low_memory=False)
test_usuarios_calificaciones = pd.read_csv('test_usuarios_calificaciones.csv', low_memory=False)
libros_df = pd.read_csv('Books.csv', low_memory=False)

# Preparar el LabelEncoder para 'ISBN'
label_encoder_book = LabelEncoder()
label_encoder_book.fit(pd.concat([train_usuarios_calificaciones['ISBN'], test_usuarios_calificaciones['ISBN']]))

# ID del usuario para la recomendación
user_id = 10

# Obtener todos los libros y filtrar los no calificados por el usuario
todos_libros = pd.concat([train_usuarios_calificaciones, test_usuarios_calificaciones])
libros_calificados = todos_libros[todos_libros['User-ID'] == user_id]['ISBN']

# Filtrar libros no calificados y presentes en el conjunto de datos de entrenamiento/prueba
libros_no_calificados = libros_df[libros_df['ISBN'].isin(label_encoder_book.classes_)]
libros_no_calificados = libros_no_calificados[~libros_no_calificados['ISBN'].isin(libros_calificados)]

# Preparar los datos para la predicción
user_input = np.array([user_id] * len(libros_no_calificados)).astype(int)
book_input = label_encoder_book.transform(libros_no_calificados['ISBN'].values)

# Realizar la predicción
predicciones = model.predict([user_input, book_input]).flatten()

# Obtener los índices de los 5 libros con mayores predicciones
indices_top5 = np.argsort(predicciones)[-5:]

# Recuperar los ISBNs de los top 5 libros
top5_libros_isbn = libros_no_calificados.iloc[indices_top5]['ISBN']

# Mapear los ISBNs a los títulos de los libros
top5_libros = libros_df[libros_df['ISBN'].isin(top5_libros_isbn)][['ISBN', 'Book-Title']]

print("Top 5 libros recomendados para el usuario 12345:")
print(top5_libros)

8443/8443 [==============================] - 10s 1ms/step
Top 5 libros recomendados para el usuario 12345:
              ISBN                                         Book-Title
16184   0316779059  The Baby Book: Everything You Need to Know Abo...
24409   1556611536  Julia's Last Hope (Oke, Janette, Women of the ...
28540   0201354934  HTML 4 for the World Wide Web, Fourth Edition:...
65320   0141001615  The Read-Aloud Handbook (Read-Aloud Handbook, ...
203227  0805420401  The Screwtape Letters: Also Includes \Screwtap...
